In [ ]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L).

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [ ]:
# read tables
%%capture
!wget https://www.dropbox.com/s/3jffjubrv2re97c/tr_mcc_codes.csv
!wget https://www.dropbox.com/s/vqjac7remvnn0rf/tr_types.csv
!wget https://www.dropbox.com/s/8q9ppkh8pgzxthy/transactions.csv
!wget https://www.dropbox.com/s/rroa45n9sookiro/gender_train.csv

tr_mcc_codes = pd.read_csv("tr_mcc_codes.csv", sep=";")
tr_types = pd.read_csv("tr_types.csv", sep=";")
transactions = pd.read_csv("transactions.csv", sep=",", nrows=1000000)
gender_train = pd.read_csv("gender_train.csv", sep=",")


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [ ]:
chosen_tr_type = transactions.sample(1000)
chosen_tr_type['tr_type']

366651    4011
77265     2010
639240    7010
555334    2010
523648    2010
          ... 
906600    1030
271823    1010
43711     2010
720203    7070
840999    7070
Name: tr_type, Length: 1000, dtype: int64

In [ ]:
pos_str = tr_types[tr_types['tr_description'].str.contains('POS')]
atm_str = tr_types[tr_types['tr_description'].str.contains('ATM')]

pos_str_sum = chosen_tr_type['tr_type'].isin(pos_str['tr_type']).values.sum()
atm_str_sum = chosen_tr_type['tr_type'].isin(atm_str['tr_type']).values.sum()

prop_chosen_lines = (pos_str_sum + atm_str_sum) / 1000
prop_chosen_lines

0.355

## Задание 2


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
freq_tr_types = transactions.groupby(['tr_type']).size()
freq_tr_types

tr_type
1000         8
1010    231117
1030    118975
1100     15476
1110    137658
         ...  
7074       505
7075        12
8100         2
8145         8
8146         3
Length: 72, dtype: int64

In [ ]:
tmp = tr_types.copy().set_index('tr_type')
tr_top10 = tmp.loc[freq_tr_types.nlargest(10).index[0:10]]
tmp_df = pd.DataFrame({'tr_freq': freq_tr_types.nlargest(10),'tr_description': tr_top10.squeeze()})
tmp_df

,tr_freq,tr_description
tr_type,,
1010,231117,Покупка. POS ТУ СБ РФ
2010,151166,Выдача наличных в АТМ Сбербанк России
7070,149006,Перевод на карту (с карты) через Мобильный бан...
1110,137658,Покупка. POS ТУ Россия
1030,118975,Оплата услуги. Банкоматы СБ РФ
2370,49830,Списание с карты на карту по операции <перевод...
7010,28972,Взнос наличных через АТМ (в своем тер.банке)
7030,26078,Перевод на карту (с карты) через АТМ (в предел...
7071,15781,Перевод на карту (с карты) через Мобильный бан...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
larg_inc = transactions[transactions.amount > 0].groupby(['customer_id'])['amount'].sum().nlargest(1)
larg_inc

customer_id
70780820    1.248115e+09
Name: amount, dtype: float64

In [ ]:
larg_spend = transactions[transactions.amount < 0].groupby(['customer_id'])['amount'].sum().nsmallest(1)
larg_spend

customer_id
70780820   -1.249952e+09
Name: amount, dtype: float64

In [ ]:
mod_inc = abs(transactions[transactions.customer_id == larg_inc.index[0]]['amount'].sum())
mod_spend = abs(transactions[transactions.customer_id == larg_spend.index[0]]['amount'].sum())
print(mod_inc)
print(mod_spend)

1837317.97999987
1837317.97999987


In [ ]:
transactions[transactions.customer_id == larg_inc.index[0]][transactions.amount > 0]

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
995568,70780820,389 11:05:02,6010,7031,4244.78,705492,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через АТМ (из одног...
995569,70780820,390 10:16:14,6010,7031,39954.84,700397,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через АТМ (из одног...
995570,70780820,390 11:35:51,6010,7031,24592.78,542607,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через АТМ (из одног...
995571,70780820,391 01:27:45,6010,7031,17150.49,871780,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через АТМ (из одног...
995572,70780820,391 05:50:09,6010,7031,17742.73,764917,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через АТМ (из одног...
...,...,...,...,...,...,...,...,...,...
997962,70780820,245 11:26:04,6010,7040,14396.32,370359,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через POS (в предел...
997963,70780820,333 02:07:47,6010,7040,19269.96,152770,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через POS (в предел...
997964,70780820,369 11:41:37,6010,7040,10398.59,150547,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через POS (в предел...
997965,70780820,374 08:16:14,6010,7040,54396.08,150318,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через POS (в предел...


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
tr_group = transactions.groupby(['tr_type'])
mean_tr_group = tr_group['amount'].mean().loc[tr_top10.index]
median_tr_group = tr_group['amount'].median().loc[tr_top10.index]
pd.DataFrame({'mean': mean_tr_group, 'median': median_tr_group})

,mean,median
tr_type,,
1010,-19784.748640,-7411.52
2010,-136077.629325,-44918.32
7070,65569.831700,11319.42
1110,-32119.330371,-11207.57
1030,-5320.980222,-2245.92
2370,-205418.249032,-44918.32
7010,276391.789596,112295.79
7030,86104.332909,13951.52
7071,66806.826623,3593.47


In [ ]:
tr_inc = transactions[transactions.customer_id == larg_inc.index[0]].groupby(['tr_type'])
tr_spend = transactions[transactions.customer_id == larg_spend.index[0]].groupby(['tr_type'])

mean_tr_inc = tr_inc['amount'].mean()
median_tr_inc = tr_inc['amount'].median()

mean_tr_spend = tr_spend['amount'].mean()
median_tr_spend = tr_spend['amount'].median()

print(pd.DataFrame({'mean': mean_tr_inc, 'median': median_tr_inc}))
print(pd.DataFrame({'mean': mean_tr_spend, 'median': median_tr_spend}))

                 mean       median
tr_type                           
2010    -2.941056e+06 -3368873.660
2330    -2.382398e+06 -2245915.770
2370    -2.218505e+06 -2245915.770
6110     1.756293e+04    10028.010
7020     1.465129e+04    10319.985
7021     2.850067e+04    28500.670
7030     1.406196e+04     8848.910
7031     1.921036e+04    12992.620
7034     1.430648e+04    14306.480
7040     2.153833e+04    14396.320
7070     1.384821e+04     8714.150
7071     1.860057e+04    12543.440
7074     2.281159e+04     9387.930
                 mean       median
tr_type                           
2010    -2.941056e+06 -3368873.660
2330    -2.382398e+06 -2245915.770
2370    -2.218505e+06 -2245915.770
6110     1.756293e+04    10028.010
7020     1.465129e+04    10319.985
7021     2.850067e+04    28500.670
7030     1.406196e+04     8848.910
7031     1.921036e+04    12992.620
7034     1.430648e+04    14306.480
7040     2.153833e+04    14396.320
7070     1.384821e+04     8714.150
7071     1.860057e+0

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
tr_gender_groups = transactions[transactions.amount < 0].groupby(['gender'])
abs(tr_gender_groups['amount'].mean()[0] - tr_gender_groups['amount'].mean()[1])

32718.054920224655

In [ ]:
tr_gender_groups = transactions[transactions.amount > 0].groupby(['gender'])
abs(tr_gender_groups['amount'].mean()[0] - tr_gender_groups['amount'].mean()[1])

63366.57104801464

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income".
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
zeros_max_income = transactions[transactions.amount > 0][transactions.gender == 0].groupby(['tr_type'])['amount'].max().sort_values(ascending=False).tail(10)
ones_max_income = transactions[transactions.amount > 0][transactions.gender == 1].groupby(['tr_type'])['amount'].max().sort_values(ascending=False).tail(10)

print(pd.DataFrame({'max_income': zeros_max_income}))
print(pd.DataFrame({'max_income': ones_max_income}))

         max_income
tr_type            
2210       68315.82
2010       44918.32
1010       24839.83
2110       22459.16
7075        6737.75
4100        5041.86
2370        2335.75
4210        2245.92
4110        2245.92
4051        1122.96
         max_income
tr_type            
7015      121279.45
2370      113194.15
2110      112295.79
2010       89836.63
7041       87590.72
6000       79280.83
8100       64682.37
1010       28803.87
4110        2245.92
2020        2245.92


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
zeros_max_income[zeros_max_income.keys().isin(ones_max_income.keys())].index

Int64Index([2010, 1010, 2110, 2370, 4110], dtype='int64', name='tr_type')

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин.
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
zeros_tr_sum = transactions[transactions.amount < 0][transactions.gender == 0].groupby(['mcc_code'])['amount'].sum()
ones_tr_sum = transactions[transactions.amount < 0][transactions.gender == 1].groupby(['mcc_code'])['amount'].sum()

print(zeros_tr_sum)
print(ones_tr_sum)

mcc_code
742      -386207.68
1711     -674897.69
1799     -261626.73
2741      -20746.42
3000   -74431645.27
           ...     
8699     -521107.48
8999   -19432057.92
9222     -361143.24
9311     -406197.45
9399    -1481389.65
Name: amount, Length: 174, dtype: float64
mcc_code
742      -132277.93
1711     -206264.90
1731     -121542.67
1799    -1129257.68
2741     -221290.08
           ...     
8999   -28643346.63
9222     -988205.17
9311     -373271.22
9399    -1810820.67
9402       -4581.67
Name: amount, Length: 175, dtype: float64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
mod_gender_mcc = (abs(zeros_tr_sum - ones_tr_sum).sort_values(ascending=False)).head(10)
mg_table = pd.merge(mod_gender_mcc, tr_mcc_codes.set_index('mcc_code'), how='left', on='mcc_code')
mg_table

,amount,mcc_description
mcc_code,,
6011,3.085862e+09,Финансовые институты — снятие наличности автом...
4829,2.499153e+09,Денежные переводы
6010,2.208580e+08,Финансовые институты — снятие наличности вручную
5541,1.554326e+08,Станции техобслуживания
5511,7.352877e+07,"Легковой и грузовой транспорт — продажа, серви..."
5812,7.037856e+07,"Места общественного питания, рестораны"
5977,4.241762e+07,Магазины косметики
5533,4.233808e+07,Автозапчасти и аксессуары
7995,4.079564e+07,Транзакции по азартным играм


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.

In [ ]:
transactions['tr_hour'] = transactions['tr_datetime'].str.extract(' (..):').astype(int)
transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_hour
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,10
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,14
...,...,...,...,...,...,...,...,...,...,...
999579,36837532,18 23:38:60,3000,6000,1297824.89,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,23
999580,6808459,206 19:00:10,3000,6000,124693.24,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,19
999581,83759123,90 22:40:57,3000,6000,218976.79,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,22
999582,89619970,104 23:39:35,3000,6000,408307.49,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ,23


In [ ]:
night_tr = transactions[transactions.amount < 0][transactions.tr_hour < 6].groupby(['gender'])['amount'].sum()
night_tr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


gender
0.0   -1.672008e+09
1.0   -2.573020e+09
Name: amount, dtype: float64